## Máster en Data Science - Series Temporales

# AutoGluon Model
Autor: Ramón Morillo Barrera

## Dataset: Madrid.csv

AutoGluon es una herramienta de aprendizaje automático automatizado que permite construir modelos avanzados de predicción de series temporales, de manera rápida y eficiente. Con su capacidad para automatizar la selección de características, la optimización de hiperparámetros y la combinación de modelos, AutoGluon facilita la creación de predicciones precisas al integrar factores como el clima, eventos especiales y patrones históricos de uso. Este enfoque permite optimizar los recursos, mejorar la experiencia del usuario y anticiparse a fluctuaciones en la demanda con un mínimo esfuerzo técnico.

In [1]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import matplotlib.pyplot as plt
from datetime import datetime as dt
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

### Importación de datos

In [2]:
df_autobus_month = pd.read_csv('../data/processed/Madrid_autobus_month.csv', index_col='Periodo')
df_metro_month = pd.read_csv('../data/processed/Madrid_metro_month.csv', index_col='Periodo')
df_combined_month = pd.read_csv('../data/processed/Madrid_combined_month.csv', index_col='Periodo')

df_autobus_trim = pd.read_csv('../data/processed/Madrid_autobus_trim.csv', index_col='Periodo')
df_metro_trim = pd.read_csv('../data/processed/Madrid_metro_trim.csv', index_col='Periodo')
df_combined_trim = pd.read_csv('../data/processed/Madrid_combined_trim.csv', index_col='Periodo')

In [3]:
df_autobus_month = df_autobus_month[ 'Total']
df_autobus_trim = df_autobus_trim['Total']
df_combined_month = df_combined_month[ 'Total']
df_combined_trim = df_combined_trim['Total']
df_metro_month = df_metro_month['Total']
df_metro_trim = df_metro_trim['Total']

In [4]:
df_autobus_month.index = pd.to_datetime(df_autobus_month.index)
df_autobus_trim.index = pd.to_datetime(df_autobus_trim.index)
df_combined_month.index = pd.to_datetime(df_combined_month.index)
df_combined_trim.index = pd.to_datetime(df_combined_trim.index)
df_metro_month.index = pd.to_datetime(df_metro_month.index)
df_metro_trim.index = pd.to_datetime(df_metro_trim.index)

df_autobus_month = df_autobus_month.resample("ME").last()
df_autobus_trim = df_autobus_trim.resample("QE").last()
df_combined_month = df_combined_month.resample("ME").last()
df_combined_trim = df_combined_trim.resample("QE").last()
df_metro_month = df_metro_month.resample("ME").last()
df_metro_trim = df_metro_trim.resample("QE").last()

***

## AutoGluonML

AutoGluonML es una biblioteca de aprendizaje automático de alto nivel que simplifica el desarrollo, la optimización y la implementación de modelos predictivos, incluyendo la predicción de series temporales. En el contexto de series temporales, AutoGluonML permite automatizar tareas complejas como la selección de características, el ajuste de hiperparámetros y la combinación de múltiples modelos (ensembles) para mejorar el rendimiento predictivo. Ofrece una API intuitiva que puede manejar datos en formato tabular con componentes temporales, y selecciona automáticamente los algoritmos más adecuados para predecir valores futuros, haciendo que el modelado de series temporales sea más accesible para usuarios sin experiencia avanzada en machine learning.

### Series Mensuales AutoGluon

In [17]:
# Crear el dataframe de entrenamiento y prueba
train_combined = df_combined_month[:'2020-12-31'].reset_index()
test_combined = df_combined_month['2021-01-01':].reset_index()

train_combined['item_id'] = 'combined'
test_combined['item_id'] = 'combined'

train_data = TimeSeriesDataFrame.from_data_frame(train_combined, id_column='item_id', timestamp_column='Periodo')
test_data = TimeSeriesDataFrame.from_data_frame(test_combined, id_column='item_id', timestamp_column='Periodo')

# Entrenar el modelo sin optimización de hiperparámetros
predictor = TimeSeriesPredictor(prediction_length=15, target='Total', freq='ME', eval_metric='MAPE')
predictor.fit(train_data, hyperparameter_tune_kwargs=None)

# Realizar predicciones
predictions = predictor.predict(test_data)

# Convert predictions to pandas Series
predictions.index = predictions.index.astype(str)
predictions_series = pd.Series(predictions['mean'], index=predictions.index.astype(str))

# Graficar la serie temporal con las predicciones
plt.figure(figsize=(12, 6))
plt.plot(train_combined['Periodo'], train_combined['Total'], label='Train')
plt.plot(test_combined['Periodo'], test_combined['Total'], label='Test')
plt.plot(predictions_series.index, predictions_series, label='Predictions')
plt.legend()
plt.show()

# Calcular y mostrar las métricas
mse = mean_squared_error(test_combined['Total'], predictions_series)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(test_combined['Total'], predictions_series)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}')

Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\RAMON\OneDrive\Escritorio\MASTER_CUNEF\SERIES TEMPORALES\PROYECTO FINAL EXAMEN\notebooks\AutogluonModels\ag-20241228_164404'


=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
GPU Count:          0
Memory Avail:       19.13 GB / 31.77 GB (60.2%)
Disk Space Avail:   721.85 GB / 947.05 GB (76.2%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 15,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'Total',
 'verbosity': 2}

Provided train_data has 108 rows, 1 time series. Median time series length is 108 (min=108, max=108). 

Provided data contains following columns:
	target: 'Total'

AutoGluon will gauge predictive performance using evaluation metric: 'MAPE'
	This metric's sign h

KeyboardInterrupt: 